<a href="https://colab.research.google.com/github/ahmeddtarekk7769/security-tools/blob/main/Password_Management_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import time
import random
import string

# --- Configuration Constants and In-Memory Database ---
# Using a dictionary as a simple, in-memory database substitute.
USER_DATABASE = {}

# --- Security Policy Definitions (Simple Rule-Based Policy) ---
PASSWORD_POLICY = {
    "MIN_LENGTH": 12,
    "REQUIRE_UPPER": True,
    "REQUIRE_LOWER": True,
    "REQUIRE_DIGIT": True,
    "REQUIRE_SPECIAL": True,
    "LOCKOUT_LIMIT": 3,      # Account locks after 3 failed attempts
    "EXPIRY_DAYS": 90,       # Password expires after 90 days (Timestamp method)
}

SPECIAL_CHARS = "!@#$%^&*()-_+=[]{}|;:,.<>?"

# --- 1. Password Strength Analyzer (Simple Rule-Based Model) ---

def check_password_strength(password):

    #Evaluates password strength based purely on defined rules (length and variety).


    feedback = []
    is_strong = True

    # 1. Length Check
    if len(password) < PASSWORD_POLICY["MIN_LENGTH"]:
        is_strong = False
        feedback.append(f"• Must be at least {PASSWORD_POLICY['MIN_LENGTH']} characters long.")

    # 2. Character Variety Checks
    if PASSWORD_POLICY["REQUIRE_UPPER"] and not re.search(r'[A-Z]', password):
        is_strong = False
        feedback.append("• Must contain at least one uppercase letter (A-Z).")

    if PASSWORD_POLICY["REQUIRE_LOWER"] and not re.search(r'[a-z]', password):
        is_strong = False
        feedback.append("• Must contain at least one lowercase letter (a-z).")

    if PASSWORD_POLICY["REQUIRE_DIGIT"] and not re.search(r'[0-9]', password):
        is_strong = False
        feedback.append("• Must contain at least one digit (0-9).")

    if PASSWORD_POLICY["REQUIRE_SPECIAL"] and not re.search(f'[{re.escape(SPECIAL_CHARS)}]', password):
        is_strong = False
        feedback.append(f"• Must contain at least one special character (e.g., {SPECIAL_CHARS}).")

    # --- Final Feedback ---
    if is_strong:
        return True, "STRONG: Password meets all security requirements."
    else:
        return False, f"WEAK: Password fails policy. Please address:\n" + '\n'.join(feedback)

# --- 2. Login System Implementation ---

def generate_random_password():
    #Generates a cryptographically strong random password that satisfies the policy.

    min_length = PASSWORD_POLICY["MIN_LENGTH"]

    # 1. Ensure all required character types are included (guaranteed compliance)
    password_parts = []
    if PASSWORD_POLICY["REQUIRE_UPPER"]:
        password_parts.append(random.choice(string.ascii_uppercase))
    if PASSWORD_POLICY["REQUIRE_LOWER"]:
        password_parts.append(random.choice(string.ascii_lowercase))
    if PASSWORD_POLICY["REQUIRE_DIGIT"]:
        password_parts.append(random.choice(string.digits))
    if PASSWORD_POLICY["REQUIRE_SPECIAL"]:
        password_parts.append(random.choice(SPECIAL_CHARS))

    # 2. Define the pool of characters for the rest of the password
    all_chars = string.ascii_letters + string.digits + SPECIAL_CHARS

    # 3. Fill the remaining length randomly
    remaining_length = min_length - len(password_parts)
    password_parts.extend(random.choices(all_chars, k=remaining_length))

    # 4. Shuffle the parts
    random.shuffle(password_parts)

    return "".join(password_parts)

def signup(username, password):
    #Registers a new user, enforcing the strong password policy and storing the password.

    if username in USER_DATABASE:
        return "Sign-up Failed: Username already exists."

    is_strong, feedback = check_password_strength(password)

    if not is_strong:
        return f"Sign-up Failed: Password is too weak. {feedback}"

    current_time = time.time()
    expiry_time = current_time + (PASSWORD_POLICY["EXPIRY_DAYS"] * 24 * 3600)

    # Store user data
    USER_DATABASE[username] = {
        "password": password, # Storing password in plain text
        "password_expiry_timestamp": expiry_time,
        "failed_attempts": 0,
        "is_locked": False,
        "valid_logins_count": 0,
    }

    return f"Sign-up Successful for user '{username}'."

def login(username, password):
    #Authenticates a user, checking for lockout, expiry, and correct password.

    if username not in USER_DATABASE:
        return "Login Failed: Invalid username or password."

    user = USER_DATABASE[username]

    # Check 1: Account Lockout Status
    if user["is_locked"]:
        return f"Login Failed: Account is locked out due to failed attempts. Please contact support."

    # Check 2: Password Expiry (Timestamp Method)
    if time.time() > user["password_expiry_timestamp"]:
        return "Login Failed: Your password has expired. Please reset your password immediately."

    # Check 3: Actual Password Verification
    if user["password"] == password: # Comparing plain text passwords
        # SUCCESSFUL LOGIN: Reset failed attempts counter
        user["failed_attempts"] = 0
        user["valid_logins_count"] += 1

        # Calculate and display time until expiry
        time_left = user["password_expiry_timestamp"] - time.time()
        days_left = time_left // (24 * 3600)
        hours_left = (time_left % (24 * 3600)) // 3600
        minutes_left = (time_left % 3600) // 60
        seconds_left = time_left % 60

        expiry_message = f"Login Successful! Welcome, {username}. Your password expires in {int(days_left)} days, {int(hours_left)} hours, {int(minutes_left)} minutes, and {int(seconds_left)} seconds."
        return expiry_message
    else:
        # FAILED LOGIN: Increment counter and check for lockout
        user["failed_attempts"] += 1

        if user["failed_attempts"] >= PASSWORD_POLICY["LOCKOUT_LIMIT"]:
            user["is_locked"] = True
            return f"Login Failed: Incorrect password. Account is now LOCKED OUT."
        else:
            remaining = PASSWORD_POLICY['LOCKOUT_LIMIT'] - user['failed_attempts']
            return f"Login Failed: Incorrect password. You have {remaining} attempts remaining before lockout."

# --- Interactive Demonstration ---

print("--- Interactive Password Management Flow ---")

# Generate password and show it
input("Press Enter to generate a random password...")
generated_password = generate_random_password()
print(f"Generated Password: {generated_password}")

# Check password strength of generated password
input("\nPress Enter to check the strength of the generated password...")
is_strong_gen, feedback_gen = check_password_strength(generated_password)
print(f"Generated Password Strength Check:")
print(feedback_gen)

# Signup - User provides password and strength is checked before signup
input("\nPress Enter to sign up a user.")
signup_username = input("Enter a username for signup: ")
while True:
    signup_password = input("Enter a password for signup: ")
    is_strong_signup, feedback_signup = check_password_strength(signup_password)
    print(f"Entered Password Strength Check:")
    print(feedback_signup)
    if is_strong_signup:
        signup_result = signup(signup_username, signup_password)
        print(f"Signup Result for '{signup_username}':")
        print(signup_result)
        break
    else:
        print("Please enter a stronger password.")


# Login - User provides username and password with multiple attempts
if 'signup_result' in locals() and signup_result.startswith("Sign-up Successful"):
    input("\nPress Enter to attempt to log in.")

    login_successful = False
    while not login_successful:
        login_username = input("Enter your username for login: ")
        login_password = input("Enter your password for login: ")
        login_result = login(login_username, login_password)
        print(login_result)

        # Check if login was successful or account is locked
        if login_result.startswith("Login Successful"):
            login_successful = True
        elif "Account is now LOCKED OUT" in login_result:
            break # Exit loop if account is locked

print("DONE")

--- Interactive Password Management Flow ---
Press Enter to generate a random password...
Generated Password: DtEt|5^|#D$X

Press Enter to check the strength of the generated password...
Generated Password Strength Check:
STRONG: Password meets all security requirements.

Press Enter to sign up a user.
Enter a username for signup: tiko
Enter a password for signup: tiko123
Entered Password Strength Check:
WEAK: Password fails policy. Please address:
• Must be at least 12 characters long.
• Must contain at least one uppercase letter (A-Z).
• Must contain at least one special character (e.g., !@#$%^&*()-_+=[]{}|;:,.<>?).
Please enter a stronger password.
Enter a password for signup: DtEt|5^|#D$X
Entered Password Strength Check:
STRONG: Password meets all security requirements.
Signup Result for 'tiko':
Sign-up Successful for user 'tiko'.

Press Enter to attempt to log in.
Enter your username for login: tik
Enter your password for login: DtEt|5^|#D$X
Login Failed: Invalid username or passw